<h1>Part 1: data cleaning and preprocessing</h1>

**Loading the necessary libraries**

In [5]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
from spacy.cli import download
import re
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import networkx as nx
import plotly.graph_objects as go
import random

**Setting seeds**

In [6]:
random.seed(42)
np.random.seed(42)

**Inspecting for Null values**

In [7]:
df = pd.read_csv("GEO_datasets_output.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   GEO ID           145 non-null    int64 
 1   Title            145 non-null    object
 2   Experiment type  145 non-null    object
 3   Summary          145 non-null    object
 4   Organism         145 non-null    object
dtypes: int64(1), object(4)
memory usage: 5.8+ KB


In [8]:
df.head()

,GEO ID,Title,Experiment type,Summary,Organism
0,200116672,In vivo molecular signatures of severe dengue ...,Expression profiling by high throughput sequen...,Dengue virus infection can result in severe sy...,Homo sapiens
1,200127893,The major risk factors for Alzheimer’s disease...,Expression profiling by high throughput sequen...,This SuperSeries is composed of the SubSeries ...,Mus musculus
2,200127892,The major risk factors for Alzheimer’s disease...,Expression profiling by high throughput sequen...,Microglia are involved in Alzheimer’s disease ...,Mus musculus
3,200127884,The major risk factors for Alzheimer’s disease...,Expression profiling by high throughput sequen...,Microglia are involved in Alzheimer’s disease ...,Mus musculus
4,200216999,Xenografted human microglia display diverse tr...,Expression profiling by high throughput sequen...,Microglia are central players in Alzheimer’s D...,Homo sapiens


**Connecting all text columns**

In [9]:
df["Connected text"] = df["Title"] + " " + df["Experiment type"] + " " + df["Summary"] + df["Organism"]
df = df[["GEO ID", "Connected text"]]
df.head()

,GEO ID,Connected text
0,200116672,In vivo molecular signatures of severe dengue ...
1,200127893,The major risk factors for Alzheimer’s disease...
2,200127892,The major risk factors for Alzheimer’s disease...
3,200127884,The major risk factors for Alzheimer’s disease...
4,200216999,Xenografted human microglia display diverse tr...


**Removing punctuation** <br>
Punctuation carries minimal semantic meaning in models and can introduce noise (e.g., we don't want to  "human" from "human.").

In [10]:
def remove_punctuation(text):
    text = re.sub(r'[^\w\s]', '', text)
    return text

df["Connected text"] = df["Connected text"].apply(remove_punctuation)
df.head()

,GEO ID,Connected text
0,200116672,In vivo molecular signatures of severe dengue ...
1,200127893,The major risk factors for Alzheimers disease ...
2,200127892,The major risk factors for Alzheimers disease ...
3,200127884,The major risk factors for Alzheimers disease ...
4,200216999,Xenografted human microglia display diverse tr...


**Removing stop words** <br>
Stop words tend to be high in frequency and carry no semantic value; thus, stop words
dilute the discriminative terms on which TF-IDF relies.

In [11]:
stop_words = set(stopwords.words("english"))

def remove_stopwords(text):
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

df["Connected text"] = df["Connected text"].apply(remove_stopwords)
df.head()

,GEO ID,Connected text
0,200116672,In vivo molecular signatures severe dengue inf...
1,200127893,The major risk factors Alzheimers disease Age ...
2,200127892,The major risk factors Alzheimers disease Age ...
3,200127884,The major risk factors Alzheimers disease Age ...
4,200216999,Xenografted human microglia display diverse tr...


**Changing the text to lowercase** <br>
Ensures consistency in term matching (e.g., we don't want to differentiate "human" from "Human"). This is standard unless case carries meaning (e.g., "us" vs "US").

In [12]:
def to_lower(text):
    return text.lower()

df["Connected text"] = df["Connected text"].apply(to_lower)
df.head()

,GEO ID,Connected text
0,200116672,in vivo molecular signatures severe dengue inf...
1,200127893,the major risk factors alzheimers disease age ...
2,200127892,the major risk factors alzheimers disease age ...
3,200127884,the major risk factors alzheimers disease age ...
4,200216999,xenografted human microglia display diverse tr...


**Lemmatizing text** <br>
Reduces the words to their base form (e.g., "medicating" -> "medicate"), so we treat strictly related terms as a single feature later on in the TF-IDF. This process is slower than stemming, but is superior in results (i.e. preserves linguistic meaning better). The dataset here is small; therefore, I have decided to use lemmatization.

In [13]:
nlp = spacy.load("en_core_web_sm")

def lemmatize_text(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc])

df["Connected text"] = df["Connected text"].apply(lemmatize_text)
df.head()

,GEO ID,Connected text
0,200116672,in vivo molecular signature severe dengue infe...
1,200127893,the major risk factor alzheimer disease age se...
2,200127892,the major risk factor alzheimer disease age se...
3,200127884,the major risk factor alzheimer disease age se...
4,200216999,xenografte human microglia display diverse tra...


<h1>Part 2: feature engineering and priliminary modelling<h2>

**TF-IDF** <br>
To preserve the meaning of multi-word phrases I have decided to configure TF-IDF for unigrams, bigrams and trigrams. 
I have capped the N-gram range at 3 to prevent TF-IDF from capturing too many random sequences and to avoid additional
compute.

In [14]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3))

tfidf_matrix = tfidf_vectorizer.fit_transform(df["Connected text"])

tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
tfidf_df

,000,000 483,000 483 unique,000 gene,000 gene regulate,000 individual,000 individual microglia,000 neurontypespecific,000 neurontypespecific regulatory,000 single,...,zygote 5hmc,zygote 5hmc target,zygote due,zygote due maternal,βamyloid,βamyloid aβ,βamyloid aβ deposition,βglucuronidase,βglucuronidase stain,βglucuronidase stain analysis
0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.034406,0.0,0.0,0.0,0.0,0.041627,0.041627,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.041627,0.041627,0.041627,0.000000,0.000000,0.000000
3,0.034406,0.0,0.0,0.0,0.0,0.041627,0.041627,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.041627,0.041627,0.041627,0.000000,0.000000,0.000000
4,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.038485,0.038485,0.038485
141,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
142,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
143,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


**PCA** <br>
To reduce the dimensionality I have used PCA. Prior to applying PCA, the data were normalised to N(0,1). 

In [15]:
scaler = StandardScaler()
tfidf_df = scaler.fit_transform(tfidf_df)

Below I will empiracally find the number of principal components for which the model yields the best results.

In [16]:
components_range = range(5, 31, 5)  # Approximately log(n), 2log(n), ..., 6log(n)

tfidf_pca_dict = {}

for n in components_range:
    pca = PCA(n_components=n)
    tfidf_pca_dict[f"tfidf_df{n}"] = pca.fit_transform(tfidf_df)
    print(f"Reduced to {n} components: {tfidf_pca_dict[f'tfidf_df{n}'].shape}")

Reduced to 5 components: (145, 5)
Reduced to 10 components: (145, 10)
Reduced to 15 components: (145, 15)
Reduced to 20 components: (145, 20)
Reduced to 25 components: (145, 25)
Reduced to 30 components: (145, 30)


In [17]:
# Define clustering algorithms
clustering_algorithms = {
    "KMeans": KMeans(n_clusters=3, random_state=42),
    "Agglomerative": AgglomerativeClustering(n_clusters=3),
    "DBSCAN": DBSCAN(eps=0.5, min_samples=5)
}

# DataFrame to store results
results = []

# Iterate over PCA-transformed datasets
for pcs, tfidf_matrix in tfidf_pca_dict.items():
    n_pcs = int(pcs.replace("tfidf_df", ""))  # Extract number of PCs from dictionary key
    
    # Set X_train as the TF-IDF matrix with n_pcs PCs
    X_train = pd.DataFrame(tfidf_matrix)

    # Apply each clustering algorithm
    for algo_name, model in clustering_algorithms.items():
        try:
            labels = model.fit_predict(X_train)
            
            # Compute clustering scores
            sil_score = silhouette_score(X_train, labels)
            db_score = davies_bouldin_score(X_train, labels)
            ch_score = calinski_harabasz_score(X_train, labels)

            # Append results
            results.append([n_pcs, algo_name, sil_score, db_score, ch_score])
        
        except Exception as e:
            print(f"Error with {algo_name} on {n_pcs} PCs: {e}")

# Display results as a DataFrame
results_df = pd.DataFrame(results, columns=["no. of pcs", "clustering algorithm", 
                                            "Silhouette Score", "Davies-Bouldin Score",                                            "Calinski-Harabasz Score"])
results_df

,no. of pcs,clustering algorithm,Silhouette Score,Davies-Bouldin Score,Calinski-Harabasz Score
0,5,KMeans,0.907835,0.051742,34.941980
1,5,Agglomerative,0.929827,0.034511,77.360560
2,5,DBSCAN,-0.073664,4.627814,0.238649
3,10,KMeans,0.828825,0.096972,18.738699
4,10,Agglomerative,0.857463,0.075617,31.799789
5,10,DBSCAN,-0.261924,7.732765,0.145132
6,15,KMeans,0.725525,0.159340,11.897688
7,15,Agglomerative,0.785087,0.119865,20.711806
8,15,DBSCAN,-0.484095,9.362649,0.050892
9,20,KMeans,0.500497,0.303650,7.930070


The best performing models are the KMeans and Agglomerative on 5 and 10 PCs. We will now find the optimal number of clusters for 5 PCs and 10 PCs on both these models.

<h1>Part 3: modelling and fine-tuning</h1>

In [18]:
# Define the list of clusters and PCA components to train on
cluster_range = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
pcs_range = [5, 10]

# Store results in a list
fine_tuning_results = []

# Loop over different numbers of PCs
for pcs in pcs_range:
    if pcs == 5:
        X_train = tfidf_pca_dict["tfidf_df5"]
    else:
        X_train = tfidf_pca_dict["tfidf_df10"]

    # Loop over different numbers of clusters
    for n_clusters in cluster_range:
        
        # Fine-tune KMeans
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        kmeans_labels = kmeans.fit_predict(X_train)

        # Compute metrics for KMeans
        kmeans_silhouette = silhouette_score(X_train, kmeans_labels)
        kmeans_db_score = davies_bouldin_score(X_train, kmeans_labels)
        kmeans_ch_score = calinski_harabasz_score(X_train, kmeans_labels)

        # Store results for KMeans
        fine_tuning_results.append(["KMeans", pcs, n_clusters, kmeans_silhouette, kmeans_db_score, kmeans_ch_score])

        # Fine-tune Agglomerative Clustering
        agglomerative = AgglomerativeClustering(n_clusters=n_clusters)
        agglomerative_labels = agglomerative.fit_predict(X_train)

        # Compute metrics for Agglomerative Clustering
        agglom_silhouette = silhouette_score(X_train, agglomerative_labels)
        agglom_db_score = davies_bouldin_score(X_train, agglomerative_labels)
        agglom_ch_score = calinski_harabasz_score(X_train, agglomerative_labels)

        # Store results for Agglomerative
        fine_tuning_results.append(["Agglomerative", pcs, n_clusters, agglom_silhouette, agglom_db_score, agglom_ch_score])

# Display the results as a DataFrame
fine_tuning_df = pd.DataFrame(fine_tuning_results, columns=["Model", "PCs", "n_clusters", "Silhouette Score", "Davies-Bouldin Score", "Calinski-Harabasz Score"])
fine_tuning_df

,Model,PCs,n_clusters,Silhouette Score,Davies-Bouldin Score,Calinski-Harabasz Score
0,KMeans,5,2,0.915942,0.053850,29.591863
1,Agglomerative,5,2,0.936066,0.036953,64.524930
2,KMeans,5,3,0.907835,0.051742,34.941980
3,Agglomerative,5,3,0.929827,0.034511,77.360560
4,KMeans,5,4,0.913975,0.042148,47.218683
5,Agglomerative,5,4,0.928976,0.029578,106.025189
6,KMeans,5,5,0.919718,0.034576,65.800154
7,Agglomerative,5,5,0.930463,0.023749,189.901315
8,KMeans,5,6,0.929606,0.019304,736.906360
9,Agglomerative,5,6,0.929606,0.019304,736.906360


The best performing model is the KMeans with 6 clusters and 5 PCs. Let's train it again.

In [26]:
kmeans = KMeans(n_clusters=6, random_state=42)
X_train = tfidf_pca_dict["tfidf_df5"]
kmeans_labels = kmeans.fit_predict(X_train)

# Compute metrics for KMeans
kmeans_silhouette = silhouette_score(X_train, kmeans_labels)
kmeans_db_score = davies_bouldin_score(X_train, kmeans_labels)
kmeans_ch_score = calinski_harabasz_score(X_train, kmeans_labels)

print(f"KMeans Scores: Silhouette={kmeans_silhouette:.6f}, DB={kmeans_db_score:.6f}, CH={kmeans_ch_score:.6f}\n")

KMeans Scores: Silhouette=0.929606, DB=0.019304, CH=736.906360



<h1>Part 4: visualisation</h1>

Let's extract the clusters and visualise them.

In [28]:
clusters = pd.DataFrame(kmeans_labels)
clusters.columns = ["Cluster"]
geo_clusters = pd.concat([df["GEO ID"], clusters], axis=1)
geo_clusters

,GEO ID,Cluster
0,200116672,0
1,200127893,0
2,200127892,0
3,200127884,0
4,200216999,0
...,...,...
140,200208265,0
141,200134646,0
142,200054362,4
143,200197916,0


api_call.py also saves the mapping of GEO IDs to PMIDs which we load below.

In [29]:
geo_to_pmid = pd.read_csv("GEO_to_PMID.csv")
geo_to_pmid

,GEO ID,PMID
0,200116672,30530648
1,200116672,31820734
2,200127893,31018141
3,200127893,38539015
4,200127892,31018141
...,...,...
163,200208265,35920937
164,200134646,37958987
165,200054362,25340342
166,200197916,37277533


In [30]:
merged = pd.merge(geo_to_pmid, geo_clusters, on="GEO ID", how="inner").drop(["GEO ID"], axis = 1)
merged

,PMID,Cluster
0,30530648,0
1,31820734,0
2,31018141,0
3,38539015,0
4,31018141,0
...,...,...
163,35920937,0
164,37958987,0
165,25340342,4
166,37277533,0


**Defining the plotting function** <br>
We will create two NetworkX/Plotly visualizations:

1. A graph where GEO IDs and clusters are nodes. A GEO ID node is connected to a cluster node if and only if the GEO ID belongs to that cluster. Hovering over a GEO ID node will display the associated PMID(s).

2. An analogous graph where PMIDs and clusters are nodes. Each PMID is connected to its cluster(s), and hovering over a PMID node will display the related GEO ID(s).

In [31]:
def create_cluster_graph(
    df,
    node_column,
    mapping_df,
    mapping_key,
    mapping_value,
    node_label_prefix,
    node_color,
    mapping_hover_label,
    title
):
    # Step 1: Create the graph
    G = nx.Graph()

    # Step 2: Add cluster nodes
    clusters = df["Cluster"].unique()
    for cluster in clusters:
        G.add_node(f"Cluster {cluster}", type="cluster")

    # Step 3: Add primary nodes (GEO ID or PMID) and connect to clusters
    for primary_node, group in df.groupby(node_column):
        G.add_node(primary_node, type="primary")
        for cluster in group["Cluster"].unique():
            G.add_edge(primary_node, f"Cluster {cluster}")

    # Step 4: Mapping primary node to secondary IDs
    node_to_secondary = mapping_df.groupby(mapping_key)[mapping_value].apply(list).to_dict()

    # Step 5: Node positions
    pos = nx.spring_layout(G, k=0.4, seed=42)

    # Step 6: Node data
    node_x, node_y, node_text, node_hovertext, node_color_list = [], [], [], [], []

    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

        if G.nodes[node]["type"] == "cluster":
            node_text.append(str(node))
            node_hovertext.append(str(node))
            node_color_list.append("orange")
        else:
            node_text.append(str(node))
            secondary_items = node_to_secondary.get(node, [])
            secondary_str = ", ".join(str(s) for s in secondary_items) if secondary_items else "None"
            hover_text = f"{node_label_prefix}: {node}<br>{mapping_hover_label}: {secondary_str}"
            node_hovertext.append(hover_text)
            node_color_list.append(node_color)

    # Step 7: Edges
    edge_x, edge_y = [], []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    # Step 8: Plotly figure
    fig = go.Figure()

    # Edges
    fig.add_trace(go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color="#888"),
        hoverinfo="none",
        mode="lines"
    ))

    # Nodes
    fig.add_trace(go.Scatter(
        x=node_x, y=node_y,
        mode="markers+text",
        text=node_text,
        hovertext=node_hovertext,
        hoverinfo="text",
        textposition="top center",
        marker=dict(
            color=node_color_list,
            size=10,
            line=dict(width=2)
        )
    ))

    fig.update_layout(
        title=title,
        showlegend=False,
        hovermode="closest",
        margin=dict(l=20, r=20, t=40, b=20),
        height=700,
        width = 1200
    )

    fig.show()


In [32]:
create_cluster_graph(
    df=geo_clusters,
    node_column="GEO ID",
    mapping_df=geo_to_pmid,
    mapping_key="GEO ID",
    mapping_value="PMID",
    node_label_prefix="GEO ID",
    node_color="lightgreen",
    mapping_hover_label="PMID(s)",
    title="GEO ID to Cluster Network Graph with PMIDs on Hover"
)


In [33]:
create_cluster_graph(
    df=merged,
    node_column="PMID",
    mapping_df=geo_to_pmid,
    mapping_key="PMID",
    mapping_value="GEO ID",
    node_label_prefix="PMID",
    node_color="skyblue",
    mapping_hover_label="GEO ID(s)",
    title="PMID to Cluster Network Graph with GEO IDs on Hover"
)